In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
#values that doesnot give any value to the paragraph eg where what when
from nltk.stem.porter import PorterStemmer
 #used to stem the words gives the root word
from sklearn.feature_extraction.text import TfidfVectorizer

 # text into numbers
from sklearn.model_selection import train_test_split
 #split data into training and testing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#download the stopwords from natural language toolkit
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))
#stopwords in english

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA PRE-PROCESSING

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving news.csv to news (1).csv


In [ ]:
df=pd.read_csv('news.csv')

In [ ]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
df.tail()

,Unnamed: 0,title,text,label
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL
6334,4330,Jeb Bush Is Suddenly Attacking Trump. Here's W...,Jeb Bush Is Suddenly Attacking Trump. Here's W...,REAL


In [ ]:
df.shape

(6335, 4)

In [ ]:
df.describe()

,Unnamed: 0
count,6335.000000
mean,5280.415627
std,3038.503953
min,2.000000
25%,2674.500000
50%,5271.000000
75%,7901.000000
max,10557.000000


In [ ]:
df.isna()

,Unnamed: 0,title,text,label
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
6330,False,False,False,False
6331,False,False,False,False
6332,False,False,False,False
6333,False,False,False,False


In [ ]:
df.isna().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [ ]:
#imputation is done if there are large missing values in the dataset that is you put the appropriate value in place is missing value
#but bcs dataset is big enough therfore dropping these values will be helpful for the computation if there is less data then dropping them can make the data bias

In [ ]:
#replacing the null values with empty string
df=df.fillna('')
#creating the same dataset but filling th enull vluesd with empty string

In [ ]:
#for the prediction we will be using two columns that is title and author so we wiill be combining the two for the prediction

In [ ]:
#merging , creating a new column that has author and title both
df['content']=df['text']+' '+df['title']

In [ ]:
print(df['content'])

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


TRAINING AND TESTING


In [ ]:
X=df.drop(columns='label',axis=1)
Y=df['label']

In [ ]:
print(X)


      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  \
0     Daniel Greenfield, a Shillman Journalism Fello...   
1     Google Pinterest Digg L

In [ ]:
print(Y)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object


Stemming


In [ ]:
# stemming is the process of reducing a word to its root word it will remove prefix and suffix
#actor, actress, acting root word here act

use of porter stemmer


In [ ]:
ps=PorterStemmer()

In [ ]:
def stemming(content):
  stem_content=re.sub('[^a-zA-Z]' ,' ',content)
  stem_content=stem_content.lower()
  stem_content=stem_content.split()
  stem_content=[ps.stem(word) for word in stem_content if not word in stopwords.words('english')]
  stem_content=' '.join(stem_content)
  return stem_content

In [ ]:
df['content']=df['content'].apply(stemming)

In [ ]:
print(df['content'])

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: content, Length: 6335, dtype: object


In [ ]:
x=df['content'].values
y=df['label'].values

In [ ]:
print(x)

['daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scandal realli attack 

In [ ]:
print(y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [ ]:
y.shape

(6335,)

In [ ]:
df['label']=df['label'].str.lower()

vectorizer


In [ ]:
vectorizer=TfidfVectorizer()
#tf(term frquency)= if a word is repeated then this tf puts the numerical value on that word
#idf(inverse document frequency)=word which is repeated doesnt have a meaning eq movie name review
vectorizer.fit(x)
x=vectorizer.transform(x)

In [ ]:
print(x)

  (0, 43422)	0.030377490785439623
  (0, 43299)	0.01823544823237207
  (0, 42991)	0.03695008224641025
  (0, 42987)	0.0240047103426603
  (0, 42960)	0.033095587951021126
  (0, 42927)	0.023617159002990093
  (0, 42925)	0.008737566380060865
  (0, 42859)	0.01579441180999399
  (0, 42799)	0.016678000766270875
  (0, 42712)	0.02838676929430383
  (0, 42711)	0.016808976313346893
  (0, 42674)	0.038267002580905846
  (0, 42632)	0.0425815201254337
  (0, 42535)	0.03488450955557637
  (0, 42480)	0.035809250475034055
  (0, 42466)	0.018399497997839874
  (0, 42387)	0.013920814295763086
  (0, 42356)	0.06194951909157974
  (0, 42250)	0.01149781838280218
  (0, 42219)	0.027015533479815255
  (0, 42143)	0.02866891016254173
  (0, 42082)	0.032730285384589844
  (0, 42034)	0.016336949950311784
  (0, 41986)	0.053305929475663996
  (0, 41954)	0.07035532559822699
  :	:
  (6334, 3921)	0.028594602233426542
  (6334, 3920)	0.02642484352841616
  (6334, 3742)	0.035328281056127245
  (6334, 3735)	0.020487351480229746
  (6334, 3457)

In [ ]:
# vectorizer=TfidfVectorizer()
# #tf(term frquency)= if a word is repeated then this tf puts the numerical value on that word
# #idf(inverse document frequency)=word which is repeated doesnt have a meaning eq movie name review
# vectorizer.fit(y)
# y=vectorizer.transform(y)

In [ ]:
print(y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


AttributeError: ignored

In [ ]:
y.shape

training and testing


In [92]:
X_train, X_test, Y_train, Y_test= train_test_split(x, y, test_size=0.2)

training the model

In [ ]:
model=LogisticRegression()

In [94]:
model.fit(X_train, Y_train)

LogisticRegression()

accuracy


In [96]:
X_train_prediction=model.predict(X_train)
training_accuracy=accuracy_score(X_train_prediction,Y_train)

In [97]:
print(training_accuracy)

0.9550118389897395


In [99]:
X_test_prediction=model.predict(X_test)
testing_accuracy=accuracy_score(X_test_prediction,Y_test)

In [100]:
print(testing_accuracy)

0.9021310181531176


In [105]:
#predciting system on colab only
news=X_test[0]
prediction= model.predict(news)
print(prediction)

if(prediction[0]=='FAKE'):
  print("The news is FAKE")
else:
  print("the news is REAL")

the news is REAL


In [106]:
print(Y_test[0])

REAL
